# Proyect 2: Álvaro Romero Díaz and Aleix Toda Mas

### Syntactic dimension
- Triples

### Lexical dimensions used:
- Words
- Lemmas

### Comparison metrics used:

- Difflib
- Tf/Idf
- Cosine distance

### Description 

Our idea is to build a vector for each pair of senteces, where we can store all results of comparison metrics and the labels, knowing if the sentences are paraphrase or not.

After that, we can create a matrix of data and use it to build a ML model.

When we have te model, we can test it with the test file provided.




In [1]:
from __future__ import division
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet_ic
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from nltk.parse.corenlp import CoreNLPDependencyParser

import warnings
import io
import unicodedata
import nltk
import re
import difflib
import math

In [2]:
wordnet_lemmatizer = WordNetLemmatizer()
vect = TfidfVectorizer(min_df=1)
WORD = re.compile(r'\w+')
stopWords = stopwords.words('english')
brown_ic = wordnet_ic.ic('ic-brown.dat')
warnings.filterwarnings('ignore')

In [3]:
#For cosine distance
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
    sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

#For cosine distance
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


#Print results
def printResults(y_true, y_pred, TN, FP, FN, TP):
    okCount = 0
    nokCount = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            okCount += 1
        else:
            nokCount += 1
    acc = (TP + TN) / float(TP + TN + FP + FN)
    reject = TN / float(TN + FP)
    accept = TP / float(TP + FN)
    print str(name)
    print "    acc= ", acc, " reject= ", reject, " accept= ", accept
    print "    Matched results:    ", okCount, "          ", round(okCount / (okCount + nokCount) * 100, 2), "%"
    print "    No matched results: ", nokCount, "          ", round(nokCount / (okCount + nokCount) * 100, 2), "%"
    print "    F1 score:  ", f1_score(y_true, y_pred, average="macro")
    print "    Precision: ", precision_score(y_true, y_pred, average="macro")
    print "    Recall:    ", recall_score(y_true, y_pred, average="macro")
    print "\n##########  ML MODEL CHANGED ########## \n"


"""
IN ORDER TO GET THE SENTENCE'S LEMMAS
"""


def LemmaString(sentence):
    words = nltk.tokenize.word_tokenize(sentence)
    tags = nltk.pos_tag(words)
    LemmaSentence = ""
    for tag in tags:
        # Special treatment for verbs
        if tag[1] == 'VBP' or tag[1] == 'VBP' or tag[1] == 'VBZ' or tag[1] == 'VBG' or tag[1] == 'VBD':
            lemma = wordnet_lemmatizer.lemmatize(tag[0], pos='v')
        else:
            lemma = wordnet_lemmatizer.lemmatize(tag[0])
        LemmaSentence += str(lemma) + " "
    return LemmaSentence


"""
FOR CLEANING PROCESS
"""

def cleanstopwordsandsymbols(sentence):
    words = nltk.tokenize.word_tokenize(sentence)
    sentence = nltk.pos_tag(words)
    csentence = sentence
    for word in sentence:
        #print str(word[0])
        #if word != "``" and word != ',' and word != "''":
        if str(word[0]) == "." or str(word[0]) == "``" or str(word[0]) == "," or str(word[0]) == "''" or str(word[0]) == "'":
            csentence.remove(word)
        else:
            for jword in stopWords:
                if str(word[0]) == str(jword):
                    csentence.remove(word)
    sentence = ""
    for word in csentence:
        sentence += " " + str(word[0])
    return sentence


### Parser

In [4]:
trainingPath = 'C:\\Users\\AlvaroRomero\\Google Drive\\MASTER\\IHLT\\IHLT-eval-framework\\train\\msr_paraphrase_train_input.txt'
testPath = 'C:\\Users\\AlvaroRomero\\Google Drive\\MASTER\\IHLT\\IHLT-eval-framework\\test\\msr_paraphrase_test_input.txt'

"""TRAINING"""
iniFile = io.open(trainingPath, "r", encoding="utf-8")
pairsOfSentencesTraining = []
for oneline in iniFile:
    pairsOfSentencesTraining.append(oneline.split('\t'))
iniFile.close()

labelsFile = io.open(trainingPath, "r", encoding="utf-8")
similarityLabelsTraining = []
for oneline in labelsFile:
    if oneline.replace("\n","").encode('utf-8').find("1") != -1:
        similarityLabelsTraining.append(1)
    else:
        similarityLabelsTraining.append(0)
labelsFile.close()

"""TEST"""
iniFile = io.open(testPath, "r", encoding="utf-8")
pairsOfSentencesTest = []
for oneline in iniFile:
    pairsOfSentencesTest.append(oneline.split('\t'))
iniFile.close()

labelsFile = io.open(testPath, "r", encoding="utf-8")
similarityLabelsTest = []
for oneline in labelsFile:
    if oneline.replace("\n","").encode('utf-8').find("1") != -1:
        similarityLabelsTest.append(1)
    else:
        similarityLabelsTest.append(0)
labelsFile.close()

### Definition of Machine Learning Models.

1. Random Forest version 1
2. Random Forest version 2
3. Random Forest version 3
4. 1-NN
5. 3-NN
6. 5-NN
7. SVM - RBF
8. SVM - Poly
9. SVM - Linear version 1
10. SVM - Linear version 2
11. Multilayer Perceptron version 1
12. Multilayer Perceptron version 2
13. Multilayer Perceptron version 3


In [5]:
"""
ML MODELS
"""

names = [
         "Random Forest 1",
         "Random Forest 2",
         "Random Forest 3",
         "KNN (1-NN)",
         "KNN (3-NN)",
         "KNN (5-NN)",
         "SVM v1",
         "SVM v2",
         "SVM v3",
         "Linear SVM",
         "MLP v1",
         "MLP v2",
         "MLP v3"]


classifiers = [
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    RandomForestClassifier(max_depth=10, n_estimators=5, max_features=3),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=2),
    KNeighborsClassifier(1),
    KNeighborsClassifier(3),
    KNeighborsClassifier(5),
    SVC(kernel='rbf', gamma=0.6, C=100.0),
    SVC(kernel='poly', degree = 4, C=100.0),
    SVC(kernel='linear', C=100.0, random_state=40),
    LinearSVC(C=100.0, random_state=42),
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 2), random_state=13, max_iter=100),
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20, 10, 5), random_state=3, max_iter=6000),
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15, 7), random_state=1, max_iter=500)
    ]

# Part 1

### Syntactic dimension: triples
For each pair of training sentences, we store the comparison metrics mentioned above. Once we have done this, we build a ML model and test it con same features of the test file.

In [6]:
parser = CoreNLPDependencyParser(url='http://localhost:9000')

definitiveTraining = []
definitiveTrainingPair = []
for sentences in pairsOfSentencesTraining:
    definitiveTraining = []
    try:
        for sentence in sentences:
            triplesSentence = []
            parse, = parser.raw_parse(sentence)
            for governor, dep, dependent in parse.triples():
                #definitive.append(governor, dep, dependent)
                aux = ''
                string_governor = '(' + governor[0].encode('ascii', 'ignore') + ',' + governor[1].encode('ascii', 'ignore') + ')'
                string_dependent = '(' + dependent[0].encode('ascii', 'ignore') + ',' + dependent[1].encode('ascii', 'ignore') + ')'
                aux += string_governor + ' ' + dep.encode('ascii', 'ignore') + ' ' + string_dependent
                triplesSentence.append(aux)
            definitiveTraining.append(triplesSentence)
        definitiveTrainingPair.append(definitiveTraining)
    except:
        print sentence

definitiveTest = []
definitiveTestPair = []
for sentences in pairsOfSentencesTest:
    definitiveTest =[]
    for sentence in sentences:
        triplesSentence = []
        parse, = parser.raw_parse(sentence)
        for governor, dep, dependent in parse.triples():
            aux = ''
            string_governor = '(' + governor[0].encode('ascii', 'ignore') + ',' + governor[1].encode('ascii', 'ignore') + ')'
            string_dependent = '(' + dependent[0].encode('ascii', 'ignore') + ',' + dependent[1].encode('ascii', 'ignore') + ')'
            aux += string_governor + ' ' + dep.encode('ascii', 'ignore') + ' ' + string_dependent
            triplesSentence.append(aux)
        definitiveTest.append(triplesSentence)
    definitiveTestPair.append(definitiveTest)


### Data for ML models

In [10]:
"""
TRIPLES TRAINING
"""
data = []
for pair in definitiveTrainingPair:
    foreachpair = []
    sentence1 = pair[0]
    sentence2 = pair[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)
    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    data.append(foreachpair)


"""
TRIPLES TEST
"""

dataTest = []
for pair in definitiveTestPair:
    foreachpair = []
    sentence1 = pair[0]
    sentence2 = pair[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf

    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])

    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)

### Results of part 1

In [11]:
print "\t-----------------------------------------------"
print "\t------------------ TRIPLES --------------------"
print "\t-----------------------------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	-----------------------------------------------
	------------------ TRIPLES --------------------
	-----------------------------------------------
 

Random Forest 1
    acc=  0.735072463768  reject=  0.996041171813  accept=  0.021645021645
    Matched results:     1268            73.51 %
    No matched results:  457            26.49 %
    F1 score:   0.444105968391
    Precision:  0.701169590643
    Recall:     0.508843096729

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.726376811594  reject=  0.945368171021  accept=  0.127705627706
    Matched results:     1253            72.64 %
    No matched results:  472            27.36 %
    F1 score:   0.517482517483
    Precision:  0.604294673607
    Recall:     0.536536899364

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.734492753623  reject=  0.988915281077  accept=  0.038961038961
    Matched results:     1267            73.45 %
    No matched results:  458            26.55 %
    F1 score: 

# Part 2

### Tests:

1. Only with words results. For each pair of sentences the stored vector is [ Difflib distance, Tf/IDF distance, Cosine distance, Paraphrase].
2. With words and lemmas. Same vector as point (1) plus the results of comparison metrics with lemmas.
3. Words' results with cleaning process.
4. Words' and lemmas' results, both with cleaning process.
5. Words', lemmas' and triples' results.
6. Words' and triples' results.

### Words 

In [12]:
"""
DATA MATRIX FOR TRAINING (ONLY WITH WORDS)
"""

data = []
for pair in pairsOfSentencesTraining:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    #difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    #Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    data.append(foreachpair)


"""
DATA MATRIX FOR TEST (ONLY WITH WORDS)
"""

dataTest = []
for pair in pairsOfSentencesTest:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    #difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    #Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)

In [14]:
print "\t \t----------------------"
print "\t \t----- Only words -----"
print "\t \t----------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	 	----------------------
	 	----- Only words -----
	 	----------------------
 

Random Forest 1
    acc=  0.747246376812  reject=  0.984164687253  accept=  0.0995670995671
    Matched results:     1289            74.72 %
    No matched results:  436            25.28 %
    F1 score:   0.512514778172
    Precision:  0.723108115513
    Recall:     0.54186589341

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.735072463768  reject=  0.956452889945  accept=  0.12987012987
    Matched results:     1268            73.51 %
    No matched results:  457            26.49 %
    F1 score:   0.524452546599
    Precision:  0.63602484472
    Recall:     0.543161509907

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.749565217391  reject=  0.988915281077  accept=  0.0952380952381
    Matched results:     1293            74.96 %
    No matched results:  432            25.04 %
    F1 score:   0.510895248097
    Precision:  0.753935419813
    Recall:     0.5420

### Words and lemmas

In [15]:
"""
DATA MATRIX FOR TRAINING (WORDS AND LEMMAS)
"""

data = []
for pair in pairsOfSentencesTraining:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    # LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    data.append(foreachpair)

"""
DATA MATRIX FOR TEST (WORDS AND LEMMAS)
"""

dataTest = []
for pair in pairsOfSentencesTest:
    # WORDS
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    #LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)


In [16]:
print "\t----------------------------"
print "\t----- Words and lemmas -----"
print "\t----------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	----------------------------
	----- Words and lemmas -----
	----------------------------
 

Random Forest 1
    acc=  0.744927536232  reject=  0.98495645289  accept=  0.0887445887446
    Matched results:     1285            74.49 %
    No matched results:  440            25.51 %
    F1 score:   0.503407449281
    Precision:  0.71524024024
    Recall:     0.536850520817

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.724637681159  reject=  0.946159936659  accept=  0.119047619048
    Matched results:     1250            72.46 %
    No matched results:  475            27.54 %
    F1 score:   0.511120060858
    Precision:  0.596548521665
    Recall:     0.532603777853

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.746086956522  reject=  0.985748218527  accept=  0.0909090909091
    Matched results:     1287            74.61 %
    No matched results:  438            25.39 %
    F1 score:   0.505664688148
    Precision:  0.723873873874
    Recal

### Words [cleaning process]

In [17]:
"""
DATA MATRIX FOR TRAINING (ONLY WITH WORDS [Cleaning process])
"""

data = []
for pair in pairsOfSentencesTraining:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')
    sentence1 = cleanstopwordsandsymbols(sentence1)
    sentence2 = cleanstopwordsandsymbols(sentence2)


    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    data.append(foreachpair)


"""
DATA MATRIX FOR TEST (ONLY WITH WORDS)
"""

dataTest = []
for pair in pairsOfSentencesTest:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')
    sentence1 = cleanstopwordsandsymbols(sentence1)
    sentence2 = cleanstopwordsandsymbols(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)


In [18]:
print "\t-----------------------------------------"
print "\t----- Only words [Cleaning process] -----"
print "\t-----------------------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	-----------------------------------------
	----- Only words [Cleaning process] -----
	-----------------------------------------
 

Random Forest 1
    acc=  0.744347826087  reject=  0.98495645289  accept=  0.0865800865801
    Matched results:     1284            74.43 %
    No matched results:  441            25.57 %
    F1 score:   0.501493765764
    Precision:  0.712332390583
    Recall:     0.535768269735

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.726376811594  reject=  0.940617577197  accept=  0.140692640693
    Matched results:     1253            72.64 %
    No matched results:  472            27.36 %
    F1 score:   0.525108253388
    Precision:  0.606906264083
    Recall:     0.540655108945

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.743768115942  reject=  0.987331749802  accept=  0.0779220779221
    Matched results:     1283            74.38 %
    No matched results:  442            25.62 %
    F1 score:   0.494766430942


### Words and lemmas [cleaning process]

In [19]:
"""
DATA MATRIX FOR TRAINING (WORDS AND LEMMAS) [Cleaning process]
"""

data = []
for pair in pairsOfSentencesTraining:
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')
    sentence1 = cleanstopwordsandsymbols(sentence1)
    sentence2 = cleanstopwordsandsymbols(sentence2)

    #difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    #Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))


    # LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    data.append(foreachpair)

"""
DATA MATRIX FOR TEST (WORDS AND LEMMAS)
"""

dataTest = []
for pair in pairsOfSentencesTest:
    #WORDS
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')
    sentence1 = cleanstopwordsandsymbols(sentence1)
    sentence2 = cleanstopwordsandsymbols(sentence2)

    #difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    #Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    #LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)



In [20]:
print "\t-----------------------------------------------"
print "\t----- Words and lemmas [Cleaning process] -----"
print "\t-----------------------------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	-----------------------------------------------
	----- Words and lemmas [Cleaning process] -----
	-----------------------------------------------
 

Random Forest 1
    acc=  0.740289855072  reject=  0.977830562154  accept=  0.0909090909091
    Matched results:     1277            74.03 %
    No matched results:  448            25.97 %
    F1 score:   0.50218246095
    Precision:  0.673111782477
    Recall:     0.534369826531

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.733333333333  reject=  0.961203483769  accept=  0.11038961039
    Matched results:     1265            73.33 %
    No matched results:  460            26.67 %
    F1 score:   0.511107441764
    Precision:  0.628538461538
    Recall:     0.535796547079

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.740289855072  reject=  0.983372921615  accept=  0.0757575757576
    Matched results:     1277            74.03 %
    No matched results:  448            25.97 %
    F1 score: 

### Words, lemmas and triples

In [21]:
"""
DATA MATRIX FOR TRAINING (WORDS, LEMMAS AND TRIPLES)
"""

data = []
for pair, pairTriples in zip(pairsOfSentencesTraining, definitiveTrainingPair):
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    # LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    #TRIPLES
    sentence1 = pairTriples[0]
    sentence2 = pairTriples[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)
    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    data.append(foreachpair)

"""
DATA MATRIX FOR TEST (WORDS,LEMMAS AND TRIPLES)
"""

dataTest = []
for pair, pairTriples in zip(pairsOfSentencesTest, definitiveTestPair):
    # WORDS
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    #LEMMAS
    sentence1 = LemmaString(sentence1)
    sentence2 = LemmaString(sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    # TRIPLES
    sentence1 = pairTriples[0]
    sentence2 = pairTriples[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf

    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])

    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)


In [22]:
print "\t-------------------------------------"
print "\t----- Words, lemmas and triples -----"
print "\t-------------------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)


	-------------------------------------
	----- Words, lemmas and triples -----
	-------------------------------------
 

Random Forest 1
    acc=  0.748405797101  reject=  0.980997624703  accept=  0.112554112554
    Matched results:     1291            74.84 %
    No matched results:  434            25.16 %
    F1 score:   0.522135044324
    Precision:  0.717787494813
    Recall:     0.546775868629

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.743768115942  reject=  0.955661124307  accept=  0.164502164502
    Matched results:     1283            74.38 %
    No matched results:  442            25.62 %
    F1 score:   0.550565175565
    Precision:  0.66672373452
    Recall:     0.560081644405

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.750724637681  reject=  0.985748218527  accept=  0.108225108225
    Matched results:     1295            75.07 %
    No matched results:  430            24.93 %
    F1 score:   0.520709485655
    Precision:

### Words and triples

In [7]:
"""
DATA MATRIX FOR TRAINING (WORDS AND TRIPLES)
"""

data = []
for pair, pairTriples in zip(pairsOfSentencesTraining, definitiveTrainingPair):
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    #TRIPLES
    sentence1 = pairTriples[0]
    sentence2 = pairTriples[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)
    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    data.append(foreachpair)

"""
DATA MATRIX FOR TEST (WORDS,LEMMAS AND TRIPLES)
"""

dataTest = []
for pair, pairTriples in zip(pairsOfSentencesTest, definitiveTestPair):
    # WORDS
    foreachpair = []
    sentence1 = unicodedata.normalize('NFKD', pair[0]).encode('ascii', 'ignore')
    sentence2 = unicodedata.normalize('NFKD', pair[1]).encode('ascii', 'ignore')

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf
    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])
    #Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))

    # TRIPLES
    sentence1 = pairTriples[0]
    sentence2 = pairTriples[1]
    sentence1.sort(key=str.lower)
    sentence2.sort(key=str.lower)
    sentence1 = '-'.join(str(x) for x in sentence1)
    sentence2 = '-'.join(str(y) for y in sentence2)

    # difflib
    foreachpair.append(difflib.SequenceMatcher(None, sentence1, sentence2).ratio())
    # Tfidf

    tfidf = vect.fit_transform([sentence1, sentence2])
    foreachpair.append((tfidf * tfidf.T).A[0][1])

    # Cosine
    vector1 = text_to_vector(sentence1)
    vector2 = text_to_vector(sentence2)
    foreachpair.append(get_cosine(vector1, vector2))
    dataTest.append(foreachpair)


In [10]:
print "\t-----------------------------"
print "\t----- Words and triples -----"
print "\t-----------------------------\n \n"
for name, clf in zip(names, classifiers):
    clf.fit(data, similarityLabelsTraining)
    TN, FP, FN, TP = confusion_matrix(similarityLabelsTest, clf.predict(dataTest)).ravel()
    printResults(similarityLabelsTest, clf.predict(dataTest), TN, FP, FN, TP)

	-----------------------------
	----- Words and triples -----
	-----------------------------
 

Random Forest 1
    acc=  0.754202898551  reject=  0.986539984165  accept=  0.119047619048
    Matched results:     1301            75.42 %
    No matched results:  424            24.58 %
    F1 score:   0.53029392272
    Precision:  0.758834946562
    Recall:     0.552793801606

##########  ML MODEL CHANGED ########## 

Random Forest 2
    acc=  0.731594202899  reject=  0.952494061758  accept=  0.127705627706
    Matched results:     1262            73.16 %
    No matched results:  463            26.84 %
    F1 score:   0.52085891742
    Precision:  0.622432160909
    Recall:     0.540099844732

##########  ML MODEL CHANGED ########## 

Random Forest 3
    acc=  0.750724637681  reject=  0.983372921615  accept=  0.114718614719
    Matched results:     1295            75.07 %
    No matched results:  430            24.93 %
    F1 score:   0.525098853707
    Precision:  0.734243783456
    Reca